# Embedding Analysis

In this notebook we explore how glove embeddings can be used to create a noisy set for training. In particular given a set of `BIO` encoded sequences, extract the positive labels, and use them to do some analysis in the embedding space/augment the set of positive labels.

## Algorithm

```
glove_embeddings <- load_glove()

gather positive labeled words (both word and phrase level)

augment set of positive labeled words
    - kNN with FAISS
    - logistic regression / SVM kernels for hyperplane
    - linear transforms/Affine Transforms

Analyze augmented set
```

### Resources

- FAISS (Facebook AI Similarity Search) for kNN style search with
    - L2 distance
    - Cosine Simlarity
- Scikit Learn: Logistic Regression
- Scikit Learn: SVM Kernels
- PyTorch: Linear Transforms

In [33]:
from typing import (
    Dict,
    List,
    Tuple,
    Callable,
    Optional,
)

import os
import sys
import random
from tqdm import tqdm
from collections import Counter

import torch
import altair as alt
import pandas as pd
import numpy as np
import nltk
import spacy

import allennlp

# local imports
import dpd
from dpd.dataset.bio_dataset import BIODataset
from dpd.utils import (
    remove_bio,
    explain_labels,
    get_words,
)

from dpd.constants import (
    CONLL2003_TRAIN,
    CONLL2003_VALID,
    CADEC_TRAIN,
    CADEC_VALID,
)

In [2]:
# Some generic constants to help throughout
from nltk.corpus import stopwords

STOP_WORDS = set(stopwords.words('english'))

GLOVE_EMBEDDING_DIR = 'data/glove.6B'
GLOVE_DIMS = [50, 100, 200, 300]

## Loading Embeddings
In this section we look to load the GLOVE word embeddings.

Input: `file_path: str`

Output: `Dict[str, np.ndarray]` word to embedding

In [3]:
# Useful TypeDefs
EmbeddingType = np.ndarray
EmbeddingSpaceType = Dict[str, EmbeddingType]

In [4]:
def get_glove_dim_file(dims: int, include_base=True) -> str:
    '''
    Given a number of dimensions, return the associated GLOVE embedding file
    
    Input: ``dims`` int, the number of dims
            ``include_base``, should include the full file path or just the file name
    Output: ``file_name`` str, the name of the associated file
    
    raises: Exception if number of dims is not available
    '''
    if dims not in GLOVE_DIMS:
        raise Exception(f'Unknown dims: {dims} only have {GLOVE_DIMS}')
    
    glove_file = f'glove.6B.{dims}d.txt'
    if include_base:
        glove_file = os.path.join(GLOVE_EMBEDDING_DIR, glove_file)
    return glove_file

def load_glove(dims: int) -> EmbeddingSpaceType:
    '''
    Given a number of dimensions load the embedding space for the associated GLOVE embedding
    
    Input: ``dims``: int the number of dimensions to use
    
    Output: ``EmbeddingSpace`` EmbeddingSpaceType, the entire embedding space embedded in the file
    '''
    glove_file = get_glove_dim_file(dims, include_base=True)
    with open(glove_file, 'r') as f:
        embedding_space = {}
        for line in tqdm(f):
            splitLine = line.split()
            word = splitLine[0]
            embedding = np.array([float(val) for val in splitLine[1:]])
            embedding_space[word] = embedding
    return embedding_space

In [5]:
glove_embeddings = load_glove(300)

400000it [00:44, 9078.14it/s]


## Loading Data
We will use our dataset readers that we implemented and load the CADEC dataset
In particular we will look at the ADR tag, however this should be generalized, so
we implement a set of functions to load our data given

- DataSet Type (CADEC, CONLL)
- Dataset Class (e.g. `ADR`, `PER`)

In [19]:
def get_dataset_files(dataset_type: str) -> Tuple[str, str]:
    if dataset_type == 'CONLL':
        return CONLL2003_TRAIN, CONLL2003_VALID
    elif dataset_type == 'CADEC':
        return CADEC_TRAIN, CADEC_VALID
    else:
        raise Exception(f'Unknown dataset: {dataset_type}')

def load_data(
    dataset_type: str,
    binary_class: Optional[str] = None,
) -> Tuple[BIODataset, BIODataset]:
    '''
    Load BIODataset for a given dataset type with the binary
    class if specified
    '''
    train_file, valid_file = get_dataset_files(dataset_type)

    train_dataset = BIODataset(
        dataset_id=0,
        file_name=train_file,
        binary_class=binary_class,
    )
    
    train_dataset.parse_file()
    
    valid_dataset = BIODataset(
        dataset_id=1,
        file_name=valid_file,
        binary_class=binary_class, 
    )
    
    valid_dataset.parse_file()
    
    return train_dataset, valid_dataset

train_data, valid_data = load_data('CADEC', 'ADR')

96867it [00:00, 343559.76it/s]
24143it [00:00, 360596.54it/s]


## Process BIO Data

Now that we have loaded all the proper data, we need to process the dataset, and in particular create two dictionaries

1. Word level
2. Phrase level

In [45]:
def bio_random_sample(data: BIODataset, sample_size: int) -> BIODataset:
    '''
    Genereate a random sample of the data
    '''
    bio_data = list(data.data)
    bio_dataset = BIODataset(
        dataset_id=2,
        file_name='temp.txt',
        binary_class=data.binary_class,
    )

    def _random_sample_array(array: List[object], size: int):
        for i in range(len(array)):
            ind = random.randint(0, len(array) - 1)
            temp = array[i]
            array[i] = array[ind]
            array[ind] = temp

        return array[:size]
    
    bio_dataset.data = bio_data[:sample_size] #_random_sample_array(bio_data, sample_size)
    
    return bio_dataset
train_data_sample = bio_random_sample(train_data, 50)

In [48]:
word_counter: Dict[str, Counter] = {'pos': Counter(), 'neg': Counter()}
phrase_counter: Counter = Counter()
for entry in train_data_sample:
    sentence, tags = entry['input'], entry['output']
    pos_words = get_words(sentence, tags, train_data_sample.binary_class)
    
    # get the negative words
    neg_words = get_words(sentence, tags, 'O')

    pos_ranges, pos_phrases = explain_labels(sentence, tags)
    for w in pos_words:
        word_counter['pos'][w] += 1

    for w in neg_words:
        word_counter['neg'][w] += 1

    for phrase in pos_phrases:
        phrase_counter[tuple(phrase)] += 1

# Analysis

Now we have everything setup

- `glove_embeddings`: this contains a map of word to numpy array (`np.ndarray`) of the glove embedding
- `train_data_sample`: a random sample of the training data
- `word_counter`: pos word to count and neg word to count
- `phrase_counter`: count of tuples of the most popular entities

Now we can use this data to determine how to best augment our dictionaries and define functions seperating them

### k-Nearest Neighbors

First lets look at how we can augment this dictionary through the embedding space with kNN.

- `L2 Distance` find the K nearest neighbors through the L2 distance
- `Cosine Similarity` find the K nearest neighbors through the cosine similarity

Algorithm:

1. Build Index
    - build a `FAISS` index
    - map word to index in glove
    - map index to word in glove
2. Search Index
    - build a `np.ndarray` query
3. Experiment with Hyper Parameters
    - different algorithms listed above
    - experiment with various values of k

#### Hypothesis

While visualizing the embedding space an help, our thought is that there may be different hyper planes that are closer than we anticipate, something like logisitc regression, SVM, or affine transforms might be better.